In [4]:
# !pip install dbnomics
# !pip install ray

In [1]:
import requests
import pandas as pd
import dbnomics
import pickle
import numpy as np
import datetime
import math 
import ray

In [2]:
final_df = pd.read_pickle('data/new_final_df.pkl')

In [109]:
# ''' 

# weo_data() function can fetch the data for Following variables

# 1) General government net lending/borrowing - GGXCNL_NGDP
# 2) General government gross debt - GGXWDG_NGDP
# 3) Inflation, average consumer prices	- PCPIPCH
# 4) Volume of exports of goods and services - TX_RPCH
# 5) Gross national savings	= NGSD_NGDP

# returns as list contains values as follows in sequence [GGXCNL_NGDP, GGXWDG_NGDP, NGSD_NGDP, PCPIPCH, TX_RPCH]

# '''
# # @ray.remote
# def weo_data(year):
#     values_ed = []
#     country_string = ''
#     for country in final_df['country_code']:
#         country_string+="+"+country
#     country_string = country_string[1:]

#     fnc = 'GGXCNL_NGDP+GGXWDG_NGDP+PCPIPCH+TX_RPCH+NGSD_NGDP'
#     sr_code = country_string+'.'+fnc

#     data = dbnomics.fetch_series("IMF", f"WEO:{year}-{10}", series_code=sr_code, max_nb_series=1000)

#     current_year_data = data.query('period=='+str(year))
#     before_1_year_data = data.query('period=='+str(year-1))
# #   before_2_year_data = data.query('period=='+str(year-2))
# #   before_3_year_data = data.query('period=='+str(year-3))

#     '''
#     country_name as key and values as list contains [GGXCNL_NGDP, GGXWDG_NGDP, NGSD_NGDP, PCPIPCH, TX_RPCH]
#     '''

#     dict_values_ggxc = {}
#     dict_values_ggxw = {}
#     dict_values_ngsd = {}
#     dict_values_pcpi = {}
#     dict_values_txrp = {}


#     vlist1 = []
#     vlist2 = []
#     vlist3 = []
#     vlist4 = []
#     vlist5 = []

#     for country in final_df.country_code:
#         cd1 = current_year_data[current_year_data['weo-country'] == country]
#         cd2 = before_1_year_data[before_1_year_data['weo-country'] == country]
# #         cd3 = before_2_year_data[before_2_year_data['weo-country'] == country]
# #         cd4 = before_3_year_data[before_3_year_data['weo-country'] == country]
#         if cd1.empty:
#             v1 = [None,None,None,None,None]
#         else:
#             v1 = list(cd1['value'].values)
#         if cd2.empty:
#             v2 = [None,None,None,None,None]
#         else:
#             v2 = list(cd2['value'].values)

#         for num, ver in enumerate([v1,v2]):
#             if ver[0] == None or str(ver[0]) == 'nan':
#                 if num ==1:
#                     dict_values_ggxc[country] = ver[0]  
#                 else:
#                     continue
#             else:
#                 dict_values_ggxc[country] = ver[0]
#                 break

#         for num, ver in enumerate([v1,v2]):
#             if ver[1] == None or str(ver[1]) == 'nan':
#                 if num ==1:
#                     dict_values_ggxw[country] = ver[1]
#                 else:
#                     continue
#             else:
#                 dict_values_ggxw[country] = ver[1]
#                 break

#         for num, ver in enumerate([v1,v2]):
#             if ver[2] == None or str(ver[2]) == 'nan':
#                 if num ==1:
#                     dict_values_ngsd[country] = ver[2]
#                 else:
#                     continue
#             else:
#                 dict_values_ngsd[country] = ver[2]
#                 break

#         for num, ver in enumerate([v1,v2]):
#             if ver[3] == None or str(ver[3]) == 'nan':
#                 if num ==1:
#                     dict_values_pcpi[country] = ver[3]
#                 else:
#                     continue
#             else:
#                 dict_values_pcpi[country] = ver[3]
#                 break

#         for num, ver in enumerate([v1,v2]):
#             if ver[4] == None or str(ver[4]) == 'nan':
#                 if num ==1:
#                     dict_values_txrp[country] = ver[4]
#                 else:
#                     continue
#             else:
#                 dict_values_txrp[country] = ver[4]
#                 break

#         vlist1.append(dict_values_ggxc[country])
#         vlist2.append(dict_values_ggxw[country])
#         vlist3.append(dict_values_ngsd[country])
#         vlist4.append(dict_values_pcpi[country])
#         vlist5.append(dict_values_txrp[country])

#     return [vlist1, vlist2, vlist3, vlist4, vlist5]

In [122]:
''' 

weo_data() function can fetch the data for Following variables

1) General government net lending/borrowing - GGXCNL_NGDP
2) General government gross debt - GGXWDG_NGDP
3) Inflation, average consumer prices	- PCPIPCH
4) Volume of exports of goods and services - TX_RPCH
5) Gross national savings	= NGSD_NGDP

returns as list contains values as follows in sequence [GGXCNL_NGDP, GGXWDG_NGDP, NGSD_NGDP, PCPIPCH, TX_RPCH]

'''
# @ray.remote
def weo_data(year, fnc):
#     values_ed = []
    country_string = ''
    for country in final_df['country_code']:
        country_string+="+"+country
    country_string = country_string[1:]

    sr_code = country_string+'.'+fnc
    month = '10'
    data = dbnomics.fetch_series("IMF", f"WEO:{year}-{month}", series_code=sr_code, max_nb_series=1000)

    current_year_data = data.query('period=='+str(year))
    before_1_year_data = data.query('period=='+str(year-1))

    '''
    country_name as key and values as list contains [GGXCNL_NGDP, GGXWDG_NGDP, NGSD_NGDP, PCPIPCH, TX_RPCH]
    '''
    dict_values = {}

    vlist = []

    for country in final_df.country_code:
        cd1 = current_year_data[current_year_data['weo-country'] == country]
        cd2 = before_1_year_data[before_1_year_data['weo-country'] == country]
#         cd3 = before_2_year_data[before_2_year_data['weo-country'] == country]
#         cd4 = before_3_year_data[before_3_year_data['weo-country'] == country]
        if cd1.empty:
            v1 = [None]
        else:
            v1 = list(cd1['value'].values)
        if cd2.empty:
            v2 = [None]
        else:
            v2 = list(cd2['value'].values)

     
        for num, ver in enumerate([v1,v2]):
            if ver[0] == None or str(ver[0]) == 'nan':
                if num ==1:
                    dict_values[country] = ver[0]
                else:
                    continue
            else:
                dict_values[country] = ver[0]
                break
        vlist.append(dict_values[country])

    return vlist

In [4]:
'''

7) External debt stocks, total (DOD, current US$) - DT.DOD.DECT.CD
8) Exports of goods and services (current US$) - NE.EXP.GNFS.CD
9) Total reserves (includes gold, current US$) - FI.RES.TOTL.CD
10) GDP per capita (constant 2010 US$) - NY.GDP.PCAP.KD
11) Imports of goods and services (current US$) - NE.IMP.GNFS.CD
12) GDP growth (annual %) - NY.GDP.MKTP.KD.ZG
13) GNI per capita (constant 2010 US$) - NY.GNP.PCAP.KD
14) GDP (current US$) - NY.GDP.MKTP.CD

'''
@ray.remote
def wb_data(year, indicatr):

    cr_yr_dict = {}
    before_1_yr_dict = {}
    values_ed = []
    for i in range(6):
        try:
            dt0 = requests.get('http://api.worldbank.org/v2/country/all/indicator/'+indicatr+'?date='+str(year)+'&format=json&page='+str(i+1)).json()
        except:
            dt0 = None
        try:
            dt1 = requests.get('http://api.worldbank.org/v2/country/all/indicator/'+indicatr+'?date='+str(year-1)+'&format=json&page='+str(i+1)).json()
        except:
            dt1 = None
            
        if dt0 != None and dt0[0]['total']!=0:
            for item in dt0[1]:
                cr_yr_dict[item['country']['id']] = item['value']
        if dt1 != None and dt1[0]['total']!=0:
            for item in dt1[1]:
                before_1_yr_dict[item['country']['id']] = item['value']

    for cntry in final_df['country_id']:
        if len(cr_yr_dict)>0 and cr_yr_dict[cntry] != None:
            values_ed.append(cr_yr_dict[cntry])
        elif len(before_1_yr_dict)>0 and before_1_yr_dict[cntry] != None:
            values_ed.append(before_1_yr_dict[cntry])
        else:
            values_ed.append(None)

    return values_ed

In [3]:
'''

15) Political Stability Estimate - PV.EST
16) Rule of Law Estimate - RL.EST
17) Govt Effectiveness Estimate - GE.EST

'''
@ray.remote
def pol_data(year, indicatr):
    cr_yr_dict = {}
    before_1_yr_dict = {}
#     before_2_yr_dict = {}
#     before_3_yr_dict = {} 
    values_ed = []
    for i in range(5):
        try:
            dt0 = requests.get('http://api.worldbank.org/v2/sources/3/country/all/series/'+indicatr+'/time/yr'+str(year)+'?format=json&page='+str(i+1)).json()
        except:
            dt0 = None
            print("No data available for {} for year {} in page {}".format(indicatr, year, i+1))
        try:
            dt1 = requests.get('http://api.worldbank.org/v2/sources/3/country/all/series/'+indicatr+'/time/yr'+str(year-1)+'?format=json&page='+str(i+1)).json()
        except:
            dt1 = None
            print("No data available for {} for year {} in page {}".format(indicatr, year-1, i+1))
   
    
        if dt0 != None:
            for item in dt0['source']['data']:
                cr_yr_dict[item['variable'][0]['id']] = item['value']
        if dt1 != None:
            for item in dt1['source']['data']:
                before_1_yr_dict[item['variable'][0]['id']] = item['value']
    
    for cntry in final_df['country_code']:
        if len(cr_yr_dict)>0 and cr_yr_dict[cntry] != None:
            values_ed.append(cr_yr_dict[cntry])
        elif len(before_1_yr_dict)>0 and before_1_yr_dict[cntry] != None:
            values_ed.append(before_1_yr_dict[cntry])
        else:
            values_ed.append(None)
    return values_ed

In [6]:
'''

15) Liquid Liabilities to GDP - GFDD.DI.05

'''
@ray.remote
def LL_GDP_data(year):
    indicatr = 'GFDD.DI.05'
    cr_yr_dict = {}
    before_1_yr_dict = {}
    values_ed = []
    for i in range(5):
        try:
            dt0 = requests.get('http://api.worldbank.org/v2/sources/32/country/all/series/GFDD.DI.05/time/yr'+str(year)+'?format=json&page='+str(i+1)).json()
        except:
            dt0 = None
            print("No data available for {} for year {} in page {}".format(indicatr, year, i+1))
        try:
            dt1 = requests.get('http://api.worldbank.org/v2/sources/32/country/all/series/GFDD.DI.05/time/yr'+str(year-1)+'?format=json&page='+str(i+1)).json()
        except:
            dt1 = None
            print("No data available for {} for year {} in page {}".format(indicatr, year-1, i+1))
    
        if dt0 != None:
            for item in dt0['source']['data']:
                cr_yr_dict[item['variable'][0]['id']] = item['value']
        if dt1 != None:
            for item in dt1['source']['data']:
                before_1_yr_dict[item['variable'][0]['id']] = item['value']
    
    for cntry in final_df['country_code']:
        if len(cr_yr_dict)>0 and cr_yr_dict[cntry] != None:
            values_ed.append(cr_yr_dict[cntry])
        elif len(before_1_yr_dict)>0 and before_1_yr_dict[cntry] != None:
            values_ed.append(before_1_yr_dict[cntry])
        else:
            values_ed.append(None)
    return values_ed

In [12]:
clms = ['fiscal_balance_gdp', 'government_grossdebt_gdp', 'inflation_data', 'exports_growth', 'savings_gdp', 'externaldebt',
               'exports', 'reserves', 'real_gdppc', 'imports', 'real_gdpgrowth', 'real_gnipc', 'nominal_gdp', 'political_stability', 'rule_of_law',
               'govt_effectiveness', 'm3_gdp'] #, 'reer']

def fetch_data(year):
#     try:
    ray.init()
    c_year = datetime.datetime.today().year 
    year = year - 1
    final_dfd = pd.read_pickle('data/new_final_df.pkl')
    if c_year >= year:
#     [fiscal_balance_gdp, government_grossdebt_gdp, savings_gdp, inflation_data, exports_growth] = weo_data(year)

#     externaldebt = wb_data(year, 'DT.DOD.DECT.CD')
#     exports = wb_data(year, 'NE.EXP.GNFS.CD')
#     political_stability = pol_data(year, 'PV.EST')

#     reserves = wb_data(year, 'FI.RES.TOTL.CD')
#     real_gdppc = wb_data(year, 'NY.GDP.PCAP.KD')
#     rule_of_law = pol_data(year, 'RL.EST')

#     imports = wb_data(year, 'NE.IMP.GNFS.CD')
#     real_gdpgrowth = wb_data(year, 'NY.GDP.MKTP.KD.ZG')
#     govt_effectiveness = pol_data(year, 'GE.EST')

#     real_gnipc = wb_data(year, 'NY.GNP.PCAP.KD')
#     nominal_gdp = wb_data(year, 'NY.GDP.MKTP.CD')

#     m3_gdp = LL_GDP_data(year)

        ret_id1 = weo_data.remote(year)
        ret_id2 = wb_data.remote(year, 'DT.DOD.DECT.CD')
        ret_id3 = wb_data.remote(year, 'NE.EXP.GNFS.CD')
        ret_id4 = pol_data.remote(year, 'PV.EST')
        ret_id5 = wb_data.remote(year, 'FI.RES.TOTL.CD')
        ret_id6 = wb_data.remote(year, 'NY.GDP.PCAP.KD')
        ret_id7 = pol_data.remote(year, 'RL.EST')
        ret_id8 = wb_data.remote(year, 'NE.IMP.GNFS.CD')
        ret_id9 = wb_data.remote(year, 'NY.GDP.MKTP.KD.ZG')
        ret_id10 = pol_data.remote(year, 'GE.EST')

        ret_id11 = wb_data.remote(year, 'NY.GNP.PCAP.KD')
        ret_id12 = wb_data.remote(year, 'NY.GDP.MKTP.CD')

        ret_id13 = LL_GDP_data.remote(year)


        [fiscal_balance_gdp, government_grossdebt_gdp, savings_gdp, inflation_data, exports_growth], externaldebt, exports, political_stability, reserves, real_gdppc, rule_of_law, imports, real_gdpgrowth, govt_effectiveness, real_gnipc, nominal_gdp, m3_gdp = ray.get([ret_id1, ret_id2, ret_id3, ret_id4, ret_id5, ret_id6, ret_id7, ret_id8, ret_id9, ret_id10, ret_id11, ret_id12, ret_id13])
    #     [fiscal_balance_gdp, government_grossdebt_gdp, savings_gdp, inflation_data, exports_growth] = ret1 
        # reer = EREER_IX_data(year)

        lt_118 = []
        ind_lst = [fiscal_balance_gdp, government_grossdebt_gdp, inflation_data, exports_growth, savings_gdp, externaldebt,
                   exports, reserves, real_gdppc, imports, real_gdpgrowth, real_gnipc, nominal_gdp, political_stability, rule_of_law,
                   govt_effectiveness, m3_gdp] #, reer]

        for nm, indctr in enumerate(ind_lst):
            if len(indctr) < 118:
                lt_118.append(clms[nm])

        for col,vals in zip(clms, ind_lst):
            if vals not in lt_118:
                final_dfd[col] = vals

        final_dfd['externaldebt/exports'] = (final_dfd['externaldebt']/final_dfd['exports'])*100
        final_dfd['reserves/import'] = (final_dfd['reserves']/final_dfd['imports'])*100
        final_dfd['reserves/gdp'] = (final_dfd['reserves']/final_dfd['nominal_gdp'])*100
        final_dfd['log_real_gnipc'] = final_dfd['real_gnipc'].apply(lambda x: math.log(x))
        final_dfd['predicted_specgrade'] = (final_dfd['log_real_gnipc']*(-5.3914)+final_dfd['savings_gdp']*(-0.1321)+final_dfd['rule_of_law']*(-4.9736)+final_dfd['government_grossdebt_gdp']*(0.1253)+46.7664)/math.sqrt(1+(7.463)**2)
        final_dfd['Speculative_Grade_Prob'] = final_dfd['predicted_specgrade'].apply(lambda x: math.exp(x)/(1+math.exp(x)))
        final_dfd['y_hat'] = final_dfd['externaldebt/exports']*( -0.0081)+final_dfd['real_gdppc']*(0.0005)+final_dfd['reserves/import']*(0.02867)+final_dfd['fiscal_balance_gdp']*(0.06526)+final_dfd['political_stability']*(1.09237)+final_dfd['default_hist']*(-0.50728)+final_dfd['govt_effectiveness']*(1.3579)+final_dfd['real_gdpgrowth']*(0.09908)+final_dfd['inflation_data']*(-0.03967)
        final_dfd['(Threshold_Yhat)/sigma']= final_dfd['y_hat'].apply(lambda x: ((-5.2355)-x)/math.sqrt(27.252))
        final_dfd['Debt_Distress_prob'] = final_dfd['(Threshold_Yhat)/sigma'].apply(lambda x: math.exp(x)/(1+math.exp(x)))
        ray.shutdown()
        return final_dfd 
    else:
        print("The Year You have Passed is in Future, Please use the Current Year or Past Year for Made this Working")
#     except:
#         print("Some Error Occured")
#         ray.shutdown()
#     finally:
#         ray.shutdown()

In [19]:
%%time
dtt = fetch_data(2020)

2020-10-31 16:29:14,791	INFO resource_spec.py:231 -- Starting Ray with 2.64 GiB memory available for workers and up to 1.32 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-10-31 16:29:15,303	INFO services.py:1193 -- View the Ray dashboard at localhost:8266


RayTaskError(ValueError): [36mray::__main__.weo_data()[39m (pid=7574, ip=192.168.0.105)
  File "python/ray/_raylet.pyx", line 479, in ray._raylet.execute_task
  File "<ipython-input-3-3ef43c9f05f8>", line 25, in weo_data
  File "/opt/anaconda3/lib/python3.7/site-packages/dbnomics/__init__.py", line 163, in fetch_series
    editor_api_base_url=editor_api_base_url)
  File "/opt/anaconda3/lib/python3.7/site-packages/dbnomics/__init__.py", line 200, in fetch_series_by_api_link
    for series_infos in iter_series_infos(api_link, max_nb_series=max_nb_series):
  File "/opt/anaconda3/lib/python3.7/site-packages/dbnomics/__init__.py", line 379, in iter_series_infos
    response_json = fetch_series_page(api_link, offset=total_nb_series)
  File "/opt/anaconda3/lib/python3.7/site-packages/dbnomics/__init__.py", line 292, in fetch_series_page
    raise ValueError("Could not fetch data from URL {!r} because: {}".format(series_page_url, message))
ValueError: Could not fetch data from URL 'https://a

(pid=7574) No data available for GFDD.DI.05 for year 2019 in page 1
(pid=7574) No data available for GFDD.DI.05 for year 2018 in page 1
(pid=7574) No data available for GFDD.DI.05 for year 2019 in page 2
(pid=7574) No data available for GFDD.DI.05 for year 2018 in page 2
(pid=7574) No data available for GFDD.DI.05 for year 2019 in page 3
(pid=7574) No data available for GFDD.DI.05 for year 2018 in page 3
(pid=7574) No data available for GFDD.DI.05 for year 2019 in page 4
(pid=7574) No data available for GFDD.DI.05 for year 2018 in page 4
(pid=7574) No data available for GFDD.DI.05 for year 2019 in page 5
(pid=7574) No data available for GFDD.DI.05 for year 2018 in page 5


In [6]:
ray.init()

2020-11-09 22:52:18,344	INFO resource_spec.py:231 -- Starting Ray with 2.2 GiB memory available for workers and up to 1.11 GiB for objects. You can adjust these settings with ray.init(memory=<bytes>, object_store_memory=<bytes>).
2020-11-09 22:52:18,994	INFO services.py:1193 -- View the Ray dashboard at localhost:8265


{'node_ip_address': '192.168.0.105',
 'raylet_ip_address': '192.168.0.105',
 'redis_address': '192.168.0.105:6379',
 'object_store_address': '/tmp/ray/session_2020-11-09_22-52-18_331933_5752/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2020-11-09_22-52-18_331933_5752/sockets/raylet',
 'webui_url': 'localhost:8265',
 'session_dir': '/tmp/ray/session_2020-11-09_22-52-18_331933_5752'}

In [25]:
dt = weo_data(2020)

In [97]:
dt[1][:10]

[7.848, 120.29, 83.288, 36.929, 96.693, 60.71, 20.113, 65.033, 41.786, 46.616]

In [96]:
tt[:10]

[7.848, 120.29, 83.288, 36.929, 96.693, 60.71, 20.113, 65.033, 41.786, 46.616]

In [107]:
'''
1) General government net lending/borrowing - GGXCNL_NGDP
2) General government gross debt - GGXWDG_NGDP
3) Inflation, average consumer prices	- PCPIPCH
4) Volume of exports of goods and services - TX_RPCH
5) Gross national savings	= NGSD_NGDP
'''

'\n1) General government net lending/borrowing - GGXCNL_NGDP\n2) General government gross debt - GGXWDG_NGDP\n3) Inflation, average consumer prices\t- PCPIPCH\n4) Volume of exports of goods and services - TX_RPCH\n5) Gross national savings\t= NGSD_NGDP\n'

In [123]:
tt = weo_data(2009,'PCPIPCH')

In [124]:
tt

[-9.346,
 14.021,
 1.653,
 2.549,
 5.576,
 2.989,
 2.232,
 12.898,
 3.957,
 3.847,
 5.284,
 2.654,
 3.0,
 0.9,
 12.971,
 4.3,
 4.847,
 7.0,
 8.351,
 4.628,
 2.043,
 -0.055,
 5.932,
 2.92,
 39.236,
 6.892,
 4.563,
 8.368,
 0.92,
 4.64,
 4.997,
 16.243,
 14.0,
 36.439,
 5.0,
 2.6,
 1.158,
 18.456,
 4.915,
 6.417,
 0.396,
 4.116,
 2.2,
 2.894,
 5.917,
 2.784,
 3.5,
 4.539,
 5.01,
 8.664,
 12.0,
 9.43,
 0.244,
 7.525,
 12.0,
 8.018,
 -0.614,
 4.65,
 0.234,
 2.467,
 7.316,
 5.0,
 4.6,
 7.702,
 2.8,
 1.398,
 9.911,
 5.5,
 5.428,
 -0.5,
 2.475,
 6.866,
 8.5,
 3.535,
 4.95,
 8.6,
 -0.1,
 9.12,
 4.8,
 11.959,
 4.3,
 13.2,
 3.323,
 20.775,
 2.299,
 3.204,
 2.837,
 8.231,
 3.367,
 2.8,
 0.0,
 5.535,
 12.27,
 11.5,
 4.5,
 11.0,
 -0.909,
 10.623,
 0.981,
 9.85,
 7.55,
 7.5,
 6.456,
 2.761,
 -1.151,
 8.014,
 7.24,
 3.5,
 6.2,
 10.575,
 14.2,
 16.275,
 7.5,
 29.45,
 7.0,
 8.429,
 7.198,
 13.989]

In [125]:
ray.shutdown()

In [22]:
len(dt[1])

IndexError: list index out of range

In [22]:
# dtt.to_pickle('2019.pkl')

In [68]:
dtt.Debt_Distress_prob.min()

0.05666380230839936

In [80]:
dtt.columns

Index(['country_id', 'country_code', 'country_name', 'default_hist', 'region',
       'fiscal_balance_gdp', 'government_grossdebt_gdp', 'inflation_data',
       'exports_growth', 'savings_gdp', 'externaldebt', 'exports', 'reserves',
       'real_gdppc', 'imports', 'real_gdpgrowth', 'real_gnipc', 'nominal_gdp',
       'political_stability', 'rule_of_law', 'govt_effectiveness', 'm3_gdp',
       'externaldebt/exports', 'reserves/import', 'reserves/gdp',
       'log_real_gnipc', 'predicted_specgrade', 'Speculative_Grade_Prob',
       'y_hat', '(Threshold_Yhat)/sigma', 'Debt_Distress_prob'],
      dtype='object')

In [93]:
dt_test = dtt[dtt.region == 'North America']

In [96]:
dt_test

,country_id,country_code,country_name,default_hist,region,fiscal_balance_gdp,government_grossdebt_gdp,inflation_data,exports_growth,savings_gdp,...,m3_gdp,externaldebt/exports,reserves/import,reserves/gdp,log_real_gnipc,predicted_specgrade,Speculative_Grade_Prob,y_hat,(Threshold_Yhat)/sigma,Debt_Distress_prob


In [94]:
dt_test[['country_name','Debt_Distress_prob']].sort_values(by=['Debt_Distress_prob'], ascending=False).dropna()

,country_name,Debt_Distress_prob


In [95]:
dt_test[['country_name','Speculative_Grade_Prob']].sort_values(by=['Speculative_Grade_Prob'], ascending=False).dropna()

,country_name,Speculative_Grade_Prob


In [25]:
dtt.country_name.to_list()

['Afghanistan',
 'Angola',
 'Albania',
 'United Arab Emirates',
 'Argentina',
 'Armenia',
 'Azerbaijan',
 'Burundi',
 'Benin',
 'Burkina Faso',
 'Bangladesh',
 'Bulgaria',
 'Bahrain',
 'Bosnia and Herzegovina',
 'Belarus',
 'Bolivia',
 'Brazil',
 'Bhutan',
 'Botswana',
 'Central African Republic',
 'Chile',
 'China',
 "Côte d'Ivoire",
 'Cameroon',
 'Democratic Republic of the Congo',
 'Republic of Congo',
 'Colombia',
 'Costa Rica',
 'Dominican Republic',
 'Algeria',
 'Ecuador',
 'Egypt',
 'Eritrea',
 'Ethiopia',
 'Fiji',
 'Gabon',
 'Georgia',
 'Ghana',
 'Guinea',
 'The Gambia',
 'Guinea-Bissau',
 'Equatorial Guinea',
 'Guatemala',
 'Guyana',
 'Honduras',
 'Croatia',
 'Haiti',
 'Hungary',
 'Indonesia',
 'India',
 'Islamic Republic of Iran',
 'Jamaica',
 'Jordan',
 'Kazakhstan',
 'Kenya',
 'Kyrgyz Republic',
 'Cambodia',
 'Kuwait',
 'Lao P.D.R.',
 'Lebanon',
 'Liberia',
 'Libya',
 'Sri Lanka',
 'Lesotho',
 'Morocco',
 'Moldova',
 'Madagascar',
 'Maldives',
 'Mexico',
 'North Macedonia',

In [26]:
dtt.Debt_Distress_prob.fillna(0).to_list()


[0.3728504519228813,
 0.2791143008868058,
 0.183842929838349,
 0.0,
 0.2498427600595613,
 0.20659063091166363,
 0.16425907171049836,
 0.49295823651919196,
 0.0,
 0.0,
 0.2910789271273844,
 0.08837582478510211,
 0.0,
 0.186605550218165,
 0.16489606623944453,
 0.27817329137389124,
 0.09781712764743755,
 0.16365654115738723,
 0.07794020939265903,
 0.48028369323673226,
 0.0,
 0.07643853948781329,
 0.0,
 0.3584781479286106,
 0.4417353842905334,
 0.3295247880989202,
 0.1695284842977434,
 0.10468761222320815,
 0.1713708061799321,
 0.14057454967497673,
 0.25006389442184096,
 0.2942752270476254,
 0.0,
 0.4641348349721628,
 0.0,
 0.1595075062675306,
 0.17707350567277966,
 0.2404600412044479,
 0.3186439074720324,
 0.30558874066353675,
 0.0,
 0.0,
 0.2357359429721818,
 0.22291977058403858,
 0.26123968870195424,
 0.0,
 0.4422410371600387,
 0.0,
 0.20189903742659138,
 0.2075371810075708,
 0.0,
 0.18202702729669554,
 0.0,
 0.10850757650296568,
 0.36100730267191666,
 0.31788695239839143,
 0.1786126938

In [40]:
dtt[['country_name','country_code','country_id']].values

array([['Afghanistan', 'AFG', 'AF'],
       ['Angola', 'AGO', 'AO'],
       ['Albania', 'ALB', 'AL'],
       ['United Arab Emirates', 'ARE', 'AE'],
       ['Argentina', 'ARG', 'AR'],
       ['Armenia', 'ARM', 'AM'],
       ['Azerbaijan', 'AZE', 'AZ'],
       ['Burundi', 'BDI', 'BI'],
       ['Benin', 'BEN', 'BJ'],
       ['Burkina Faso', 'BFA', 'BF'],
       ['Bangladesh', 'BGD', 'BD'],
       ['Bulgaria', 'BGR', 'BG'],
       ['Bahrain', 'BHR', 'BH'],
       ['Bosnia and Herzegovina', 'BIH', 'BA'],
       ['Belarus', 'BLR', 'BY'],
       ['Bolivia', 'BOL', 'BO'],
       ['Brazil', 'BRA', 'BR'],
       ['Bhutan', 'BTN', 'BT'],
       ['Botswana', 'BWA', 'BW'],
       ['Central African Republic', 'CAF', 'CF'],
       ['Chile', 'CHL', 'CL'],
       ['China', 'CHN', 'CN'],
       ["Côte d'Ivoire", 'CIV', 'CI'],
       ['Cameroon', 'CMR', 'CM'],
       ['Democratic Republic of the Congo', 'COD', 'CD'],
       ['Republic of Congo', 'COG', 'CG'],
       ['Colombia', 'COL', 'CO'],
       ['Co

In [48]:
for i in dtt[['country_code']].values:
    print(i[0])

AFG
AGO
ALB
ARE
ARG
ARM
AZE
BDI
BEN
BFA
BGD
BGR
BHR
BIH
BLR
BOL
BRA
BTN
BWA
CAF
CHL
CHN
CIV
CMR
COD
COG
COL
CRI
DOM
DZA
ECU
EGY
ERI
ETH
FJI
GAB
GEO
GHA
GIN
GMB
GNB
GNQ
GTM
GUY
HND
HRV
HTI
HUN
IDN
IND
IRN
JAM
JOR
KAZ
KEN
KGZ
KHM
KWT
LAO
LBN
LBR
LBY
LKA
LSO
MAR
MDA
MDG
MDV
MEX
MKD
MLI
MMR
MNG
MOZ
MRT
MWI
MYS
NAM
NER
NGA
NIC
NPL
OMN
PAK
PAN
PER
PHL
PNG
POL
PRY
QAT
ROU
RUS
RWA
SAU
SDN
SEN
SLE
SLV
SRB
SUR
SYR
TCD
TGO
THA
TJK
TTO
TUN
TUR
TZA
UGA
UKR
URY
VEN
VNM
YEM
ZAF
ZMB


In [49]:
dt = pd.read_pickle('data/new_final_df.pkl')

In [50]:
dt

,country_id,country_code,country_name,default_hist
0,AF,AFG,Afghanistan,0
1,AO,AGO,Angola,0
2,AL,ALB,Albania,0
3,AE,ARE,United Arab Emirates,0
4,AR,ARG,Argentina,1
...,...,...,...,...
113,VE,VEN,Venezuela,1
114,VN,VNM,Vietnam,0
115,YE,YEM,Yemen,0
116,ZA,ZAF,South Africa,0


In [51]:
dt['region'] = ["South Asia","Sub-Saharan Africa","Europe and Central Asia","Middle East and North Africa","Latin America & the Caribbean","Europe and Central Asia","Europe and Central Asia","Sub-Saharan Africa","Sub-Saharan Africa","Sub-Saharan Africa","South Asia","Europe and Central Asia","Middle East and North Africa","Europe and Central Asia","Europe and Central Asia","Latin America & the Caribbean","Latin America & the Caribbean","South Asia","Sub-Saharan Africa","Sub-Saharan Africa","Latin America & the Caribbean","East Asia and Pacific","Sub-Saharan Africa","Sub-Saharan Africa","Sub-Saharan Africa","Sub-Saharan Africa","Latin America & the Caribbean","Latin America & the Caribbean","Latin America & the Caribbean","Middle East and North Africa","Latin America & the Caribbean","Middle East and North Africa","Sub-Saharan Africa","Sub-Saharan Africa","East Asia and Pacific","Sub-Saharan Africa","Europe and Central Asia","Sub-Saharan Africa","Sub-Saharan Africa","Sub-Saharan Africa","Sub-Saharan Africa","Sub-Saharan Africa","Latin America & the Caribbean","Latin America & the Caribbean","Latin America & the Caribbean","Europe and Central Asia","Latin America & the Caribbean","Europe and Central Asia","East Asia and Pacific","South Asia","Middle East and North Africa","Latin America & the Caribbean","Middle East and North Africa","Europe and Central Asia","Sub-Saharan Africa","Europe and Central Asia","East Asia and Pacific","Middle East and North Africa","East Asia and Pacific","Middle East and North Africa","Sub-Saharan Africa","Middle East and North Africa","South Asia","Sub-Saharan Africa","Middle East and North Africa","Europe and Central Asia","Sub-Saharan Africa","South Asia","Latin America & the Caribbean","Europe and Central Asia","Sub-Saharan Africa","East Asia and Pacific","East Asia and Pacific","Sub-Saharan Africa","Sub-Saharan Africa","Sub-Saharan Africa","East Asia and Pacific","Sub-Saharan Africa","Sub-Saharan Africa","Sub-Saharan Africa","Latin America & the Caribbean","South Asia","Middle East and North Africa","South Asia","Latin America & the Caribbean","Latin America & the Caribbean","East Asia and Pacific","East Asia and Pacific","Europe and Central Asia","Latin America & the Caribbean","Middle East and North Africa","Europe and Central Asia","Europe and Central Asia","Sub-Saharan Africa","Middle East and North Africa","Sub-Saharan Africa","Sub-Saharan Africa","Sub-Saharan Africa","Latin America & the Caribbean","Europe and Central Asia","Latin America & the Caribbean","Middle East and North Africa","Sub-Saharan Africa","Sub-Saharan Africa","East Asia and Pacific","Europe and Central Asia","Latin America & the Caribbean","Middle East and North Africa","Europe and Central Asia","Sub-Saharan Africa","Sub-Saharan Africa","Europe and Central Asia","Latin America & the Caribbean","Latin America & the Caribbean","East Asia and Pacific","Middle East and North Africa","Sub-Saharan Africa","Sub-Saharan Africa"]

In [92]:
dt[dt.country_name == 'Nepal']

,country_id,country_code,country_name,default_hist,region
81,NP,NPL,Nepal,0,South Asia


In [63]:
dt.to_pickle('data/new_final_df2.pkl')

In [29]:
year = 2019

In [32]:
values_ed = []

In [33]:
country_string = ''

In [34]:
for country in final_df['country_code']:
        country_string+="+"+country

In [35]:
country_string = country_string[1:]

In [90]:
test_str = country_string+'.'+'GGXWDG_NGDP'

In [92]:
test_str

'AFG+AGO+ALB+ARE+ARG+ARM+AZE+BDI+BEN+BFA+BGD+BGR+BHR+BIH+BLR+BOL+BRA+BTN+BWA+CAF+CHL+CHN+CIV+CMR+COD+COG+COL+CRI+DOM+DZA+ECU+EGY+ERI+ETH+FJI+GAB+GEO+GHA+GIN+GMB+GNB+GNQ+GTM+GUY+HND+HRV+HTI+HUN+IDN+IND+IRN+JAM+JOR+KAZ+KEN+KGZ+KHM+KWT+LAO+LBN+LBR+LBY+LKA+LSO+MAR+MDA+MDG+MDV+MEX+MKD+MLI+MMR+MNG+MOZ+MRT+MWI+MYS+NAM+NER+NGA+NIC+NPL+OMN+PAK+PAN+PER+PHL+PNG+POL+PRY+QAT+ROU+RUS+RWA+SAU+SDN+SEN+SLE+SLV+SRB+SUR+SYR+TCD+TGO+THA+TJK+TTO+TUN+TUR+TZA+UGA+UKR+URY+VEN+VNM+YEM+ZAF+ZMB.GGXWDG_NGDP'

In [117]:
df =dbnomics.fetch_series("IMF", "WEO:2020-10", series_code=test_str, max_nb_series=1000)

In [119]:
df[df['original_period'] == '2020']

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,weo-country,weo-subject,unit,WEO Country,WEO Subject,Unit
40,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),AFG.GGXWDG_NGDP.pcent_gdp,Afghanistan – General government gross debt (G...,2020,2020-01-01,7.848,7.848,AFG,GGXWDG_NGDP,pcent_gdp,Afghanistan,General government gross debt,Percent of GDP
40,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),AGO.GGXWDG_NGDP.pcent_gdp,Angola – General government gross debt (GGXWDG...,2020,2020-01-01,120.29,120.29,AGO,GGXWDG_NGDP,pcent_gdp,Angola,General government gross debt,Percent of GDP
40,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ALB.GGXWDG_NGDP.pcent_gdp,Albania – General government gross debt (GGXWD...,2020,2020-01-01,83.288,83.288,ALB,GGXWDG_NGDP,pcent_gdp,Albania,General government gross debt,Percent of GDP
40,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ARE.GGXWDG_NGDP.pcent_gdp,United Arab Emirates – General government gros...,2020,2020-01-01,36.929,36.929,ARE,GGXWDG_NGDP,pcent_gdp,United Arab Emirates,General government gross debt,Percent of GDP
40,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ARG.GGXWDG_NGDP.pcent_gdp,Argentina – General government gross debt (GGX...,2020,2020-01-01,96.693,96.693,ARG,GGXWDG_NGDP,pcent_gdp,Argentina,General government gross debt,Percent of GDP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),VEN.GGXWDG_NGDP.pcent_gdp,Venezuela – General government gross debt (GGX...,2020,2020-01-01,NA,NaN,VEN,GGXWDG_NGDP,pcent_gdp,Venezuela,General government gross debt,Percent of GDP
40,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),VNM.GGXWDG_NGDP.pcent_gdp,Vietnam – General government gross debt (GGXWD...,2020,2020-01-01,46.623,46.623,VNM,GGXWDG_NGDP,pcent_gdp,Vietnam,General government gross debt,Percent of GDP
40,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),YEM.GGXWDG_NGDP.pcent_gdp,Yemen – General government gross debt (GGXWDG_...,2020,2020-01-01,81.651,81.651,YEM,GGXWDG_NGDP,pcent_gdp,Yemen,General government gross debt,Percent of GDP
40,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ZAF.GGXWDG_NGDP.pcent_gdp,South Africa – General government gross debt (...,2020,2020-01-01,78.821,78.821,ZAF,GGXWDG_NGDP,pcent_gdp,South Africa,General government gross debt,Percent of GDP


In [ ]:


fnc = 'GGXCNL_NGDP+GGXWDG_NGDP+PCPIPCH+TX_RPCH+NGSD_NGDP'
sr_code = country_string+'.'+fnc

data = dbnomics.fetch_series("IMF", "WEO", series_code=sr_code, max_nb_series=1000)



In [111]:
df = dbnomics.fetch_series_by_api_link('https://api.db.nomics.world/v22/series/IMF/WEO:2020-10/.GGXCNL_NGDP+GGXWDG_NGDP+PCPIPCH+TX_RPCH+NGSD_NGDP?observations=1&offset=0',max_nb_series=1000)

In [112]:
df[df['original_period'] == '2019']

,@frequency,provider_code,dataset_code,dataset_name,series_code,series_name,original_period,period,original_value,value,weo-country,weo-subject,unit,WEO Country,WEO Subject,Unit
39,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ABW.GGXCNL_NGDP.pcent_gdp,Aruba – General government net lending/borrowi...,2019,2019-01-01,0.39,0.39,ABW,GGXCNL_NGDP,pcent_gdp,Aruba,General government net lending/borrowing,Percent of GDP
39,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ABW.GGXWDG_NGDP.pcent_gdp,Aruba – General government gross debt (GGXWDG_...,2019,2019-01-01,81.286,81.286,ABW,GGXWDG_NGDP,pcent_gdp,Aruba,General government gross debt,Percent of GDP
39,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ABW.NGSD_NGDP.pcent_gdp,Aruba – Gross national savings – Percent of GDP,2019,2019-01-01,17,17,ABW,NGSD_NGDP,pcent_gdp,Aruba,Gross national savings,Percent of GDP
39,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ABW.PCPIPCH.pcent_change,"Aruba – Inflation, average consumer prices (PC...",2019,2019-01-01,4.257,4.257,ABW,PCPIPCH,pcent_change,Aruba,"Inflation, average consumer prices",Percent change
39,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ABW.TX_RPCH.pcent_change,Aruba – Volume of exports of goods and service...,2019,2019-01-01,NA,None,ABW,TX_RPCH,pcent_change,Aruba,Volume of exports of goods and services,Percent change
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ZWE.GGXCNL_NGDP.pcent_gdp,Zimbabwe – General government net lending/borr...,2019,2019-01-01,-1.649,-1.649,ZWE,GGXCNL_NGDP,pcent_gdp,Zimbabwe,General government net lending/borrowing,Percent of GDP
39,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ZWE.GGXWDG_NGDP.pcent_gdp,Zimbabwe – General government gross debt (GGXW...,2019,2019-01-01,10.806,10.806,ZWE,GGXWDG_NGDP,pcent_gdp,Zimbabwe,General government gross debt,Percent of GDP
39,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ZWE.NGSD_NGDP.pcent_gdp,Zimbabwe – Gross national savings – Percent of...,2019,2019-01-01,NA,None,ZWE,NGSD_NGDP,pcent_gdp,Zimbabwe,Gross national savings,Percent of GDP
39,annual,IMF,WEO:2020-10,WEO by countries (2020-10 release),ZWE.PCPIPCH.pcent_change,"Zimbabwe – Inflation, average consumer prices ...",2019,2019-01-01,255.292,255.292,ZWE,PCPIPCH,pcent_change,Zimbabwe,"Inflation, average consumer prices",Percent change


In [120]:
c_year = datetime.datetime.today().year 

In [131]:
datetime.datetime.today().month >= 10 and datetime.datetime.today().month < 4

False